## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,hobart,AU,-42.8794,147.3294,57.06,81,40,5.75,scattered clouds
1,1,torbay,CA,47.6666,-52.7314,39.88,48,75,23.02,broken clouds
2,2,vaini,TO,-21.2000,-175.2000,75.36,94,40,4.61,scattered clouds
3,3,pevek,RU,69.7008,170.3133,-5.71,82,4,3.71,clear sky
4,4,ushuaia,AR,-54.8000,-68.3000,53.22,54,40,25.32,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(5)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,vaini,TO,-21.2000,-175.2000,75.36,94,40,4.61,scattered clouds
6,6,rikitea,PF,-23.1203,-134.9692,77.83,72,10,4.03,clear sky
14,14,mahebourg,MU,-20.4081,57.7000,78.26,94,40,8.05,scattered clouds
21,21,iaciara,BR,-14.0958,-46.6317,82.26,70,98,2.53,overcast clouds
25,25,georgetown,MY,5.4112,100.3354,82.33,93,20,3.44,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                183
City                   183
Country                181
Lat                    183
Lng                    183
Max Temp               183
Humidity               183
Cloudiness             183
Wind Speed             183
Current Description    183
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,vaini,TO,-21.2000,-175.2000,75.36,94,40,4.61,scattered clouds
6,6,rikitea,PF,-23.1203,-134.9692,77.83,72,10,4.03,clear sky
14,14,mahebourg,MU,-20.4081,57.7000,78.26,94,40,8.05,scattered clouds
21,21,iaciara,BR,-14.0958,-46.6317,82.26,70,98,2.53,overcast clouds
25,25,georgetown,MY,5.4112,100.3354,82.33,93,20,3.44,few clouds


In [7]:
# clean up the DataFrame by dropna() funtion
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.count()
# make a copy for a new DataFrame 
hotel_df = clean_cities_df[["City", "Country","Max Temp",  "Current Description","Lat", "Lng"]].copy()
hotel_df['Hotel Name'] = ''

# search nearest hotel name by using google place API 
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params = {'key': g_key, 'type':'lodging', 'radius' : 5000}
    params['location'] = f"{lat},{lng}"
    try: 
        r_hotels = requests.get(base_url, params= params).json()

        hotel_df.loc[index, 'Hotel Name'] = r_hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')



Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [8]:
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,rikitea,PF,77.83,clear sky,-23.1203,-134.9692,People ThankYou
14,mahebourg,MU,78.26,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
21,iaciara,BR,82.26,overcast clouds,-14.0958,-46.6317,Hotel Matinha
25,georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang


In [9]:
# 8a. Create the output File (CSV)
clean_hotel_df.to_csv("WeatherPy_vacation.csv")
clean_hotel_df.to_csv(("WeatherPy_vacation.csv"), index_label=("City_ID"))


In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))